In [1]:
%pip install langchain pinecone

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install langchain-pinecone

   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   --- ----------------------------------- 20.5/215.9 kB 640.0 kB/s eta 0:00:01
   -------------- ------------------------ 81.9/215.9 kB 907.3 kB/s eta 0:00:01
   ---------------------------------------  215.0/215.9 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 215.9/215.9 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain import PromptTemplate        
from langchain.chains import RetrievalQA                
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter      #for creating chunks
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers 
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

In [24]:
import os

In [25]:
os.environ['PINECONE_API_KEY'] = 'dfad884a-1a06-493c-850a-1de28db1b2dc'

In [5]:
# PINCONE_API_KEY="dfad884a-1a06-493c-850a-1de28db1b2dc"
# PINCONE_API_ENV=""
pc = Pinecone(api_key="dfad884a-1a06-493c-850a-1de28db1b2dc")

index_name = "mchatbot"

if index_name not in pc.list_indexes().names():
  pc.create_index(
  name="mchatbot",
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
) 

In [6]:
# Data extraction from pdf

def load_pdf(data):
    loader= DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents= loader.load()

    return documents

In [7]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
extracted_data = load_pdf("data/")

In [9]:
# extracted_data

In [10]:
# Creating text Chunks so that the entire pdf is splitted into smaller texts

def text_split(extracted_data):
    # keeping size of each chunk to be 500 with overlap b/w chunks as 20
    text_splitter= RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap = 20)
    text_chunks= text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("No of chunks formed:", len(text_chunks))

No of chunks formed: 10484


In [12]:
# text_chunks

In [13]:
# Convert these chunks made into vector

# Download the embeddig model

def download_hugging_face_embedding():
    embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings= download_hugging_face_embedding()

c:\Users\rohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings_val = model.encode(sentences)
print(embeddings_val)

c:\Users\rohit\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[ 6.76568970e-02  6.34958893e-02  4.87131625e-02  7.93049783e-02
   3.74480598e-02  2.65280833e-03  3.93749252e-02 -7.09846523e-03
   5.93614690e-02  3.15369815e-02  6.00980707e-02 -5.29052205e-02
   4.06067818e-02 -2.59308573e-02  2.98427679e-02  1.12689380e-03
   7.35148266e-02 -5.03819436e-02 -1.22386597e-01  2.37028059e-02
   2.97265295e-02  4.24768925e-02  2.56337542e-02  1.99516653e-03
  -5.69190606e-02 -2.71597914e-02 -3.29036117e-02  6.60248548e-02
   1.19007125e-01 -4.58791181e-02 -7.26214349e-02 -3.25840451e-02
   5.23413755e-02  4.50552925e-02  8.25304631e-03  3.67023908e-02
  -1.39415385e-02  6.53919131e-02 -2.64272336e-02  2.06404657e-04
  -1.36644067e-02 -3.62810045e-02 -1.95043534e-02 -2.89738458e-02
   3.94270346e-02 -8.84090886e-02  2.62422930e-03  1.36714159e-02
   4.83062528e-02 -3.11564859e-02 -1.17329188e-01 -5.11689931e-02
  -8.85287523e-02 -2.18962859e-02  1.42986355e-02  4.44168150e-02
  -1.34815034e-02  7.43392408e-02  2.66382936e-02 -1.98762082e-02
   1.79190

In [17]:
query_result = embeddings.embed_query("Hello World!")
print("Length of embedding:", len(query_result))

Length of embedding: 384


In [27]:
vectorstore = PineconeVectorStore.from_documents(
        documents=text_chunks,
        index_name=index_name,
        embedding=embeddings
    )

In [28]:
vectorstore